In [ ]:
!pip install datasets optuna scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requi

In [ ]:
import gdown

file_id = '1Y9QFfJRUIEbrvLtQRZgX6W4O3z1IzDhc'
gdown.download(f'https://drive.google.com/uc?id={file_id}', '/content/DL_train_test_data.xlsx', quiet=False)


Downloading...
From: https://drive.google.com/uc?id=1Y9QFfJRUIEbrvLtQRZgX6W4O3z1IzDhc
To: /content/DL_train_test_data.xlsx
100%|██████████| 25.9M/25.9M [00:00<00:00, 30.1MB/s]


'/content/DL_train_test_data.xlsx'

In [ ]:
# Load data
from transformers import AutoTokenizer, BertForSequenceClassification
from torch.nn import CrossEntropyLoss
from sklearn.metrics import classification_report, accuracy_score, f1_score
from torch.utils.data import Dataset, DataLoader
import torch
import pandas as pd
import optuna
from tqdm.notebook import tqdm
import numpy as np
from torch.optim import AdamW


model_name = 'bert-base-uncased'
df = pd.read_excel('DL_train_test_data.xlsx')

# Encode severity levels
df['severity_label'] = df['severity_code']
print(df['severity_label'].unique())  # Ensure labels range from 0 to 6

# Combine short and long descriptions into a single text field
df['text'] = df['short_description'].fillna('') + ' ' + df['long_description'].fillna('')

# Tokenize data
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
tokenized_data = tokenizer(
    df['text'].tolist(),
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
)

[2 6 1 4 5 0]


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:

class BugDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = {k: torch.tensor(v) for k, v in encodings.items()}
        self.labels = torch.tensor(labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.encodings['input_ids'][idx],
            'attention_mask': self.encodings['attention_mask'][idx],
            'labels': self.labels[idx]
        }

    def __len__(self):
        return len(self.labels)


labels = df['severity_label'].values
dataset = BugDataset(tokenized_data, labels)

# Split dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=7)
model.to(device)

<ipython-input-4-5a63d24080c8>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.encodings = {k: torch.tensor(v) for k, v in encodings.items()}


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
'''
OG Code for run 1 and 2. Only epoch, batchsize and learning rate are changed

'''


from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.utils.class_weight import compute_class_weight
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import WeightedRandomSampler
from torch.optim import AdamW
from tqdm import tqdm


# Training parameters
epochs = 10
# batch_size = best_hyperparams['batch_size']
# learning_rate = best_hyperparams['learning_rate']
batch_size = 16
learning_rate = 1e-4

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
optimizer = AdamW(model.parameters(), lr=learning_rate)
loss_fn = CrossEntropyLoss()

# Training loop
for epoch in range(epochs):
    model.train()
    train_loss = 0
    for batch in tqdm(train_loader):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        train_loss += loss.item()
        loss.backward()
        optimizer.step()

    avg_train_loss = train_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{epochs} | Train Loss: {avg_train_loss}")

    # Validation
    model.eval()
    val_loss = 0
    preds, true_labels = [], []
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += outputs.loss.item()
            preds.extend(torch.argmax(outputs.logits, dim=1).cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    # Compute metrics
    avg_val_loss = val_loss / len(val_loader)
    accuracy = accuracy_score(true_labels, preds)
    f1 = f1_score(true_labels, preds, average='weighted')  # Use 'weighted' for imbalanced classes

    print(f"Validation Loss: {avg_val_loss:.4f}")
    print(f"Validation Accuracy: {accuracy:.4f}")
    print(f"Validation F1 Score: {f1:.4f}")
    print(classification_report(true_labels, preds))


  0%|          | 0/2568 [00:00<?, ?it/s]

<ipython-input-9-3f123643710e>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


KeyboardInterrupt: 

In [ ]:
# Save the model
torch.save(model.state_dict(), 'optimized_model.pth')
from google.colab import files
files.download('optimized_model.pth')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# """
# Key Changes
# Weighted Loss Function: Handles class imbalance.
# WeightedRandomSampler: Balances training samples dynamically.
# Learning Rate Scheduler: Dynamically reduces the learning rate.
# Early Stopping: Stops training if validation performance plateaus.
# Evaluation Enhancements: Logs class-wise metrics and confusion matrix

# """

# from sklearn.metrics import ConfusionMatrixDisplay
# from sklearn.utils.class_weight import compute_class_weight
# from torch.optim.lr_scheduler import StepLR
# from torch.utils.data import WeightedRandomSampler
# from torch.optim import AdamW




# # Training parameters
# epochs = 10
# batch_size = 16
# learning_rate = 2e-5

# train_labels = [labels[idx] for idx in train_dataset.indices]  # Extract labels for training dataset
# class_sample_counts = np.bincount(train_labels)
# weights = 1.0 / (class_sample_counts.astype(float) + 1)
# weights = np.nan_to_num(weights, posinf=1.0)  # Avoid inf
# sample_weights = np.array([weights[label] for label in train_labels])

# sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)

# train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler)
# val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


# # Class weights for loss
# class_weights = torch.tensor(compute_class_weight(class_weight='balanced', classes=np.unique(labels), y=labels)).float().to(device)
# loss_fn = CrossEntropyLoss(weight=class_weights)


# # Optimizer and Scheduler
# optimizer = AdamW(model.parameters(), lr=learning_rate)
# scheduler = StepLR(optimizer, step_size=2, gamma=0.1)

# # Training loop with early stopping
# best_val_loss = float('inf')
# patience, patience_counter = 3, 0

# # Validate training and validation dataset indices

# for epoch in range(epochs):
#     model.train()
#     train_loss = 0
#     for batch in tqdm(train_loader):
#         optimizer.zero_grad()
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         labels = batch['labels'].to(device)
#         outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
#         loss = outputs.loss
#         train_loss += loss.item()
#         loss.backward()
#         optimizer.step()

#     avg_train_loss = train_loss / len(train_loader)
#     print(f"Epoch {epoch + 1}/{epochs} | Train Loss: {avg_train_loss:.4f}")

#     # Validation
#     model.eval()
#     val_loss = 0
#     preds, true_labels = [], []
#     with torch.no_grad():
#         for batch in tqdm(val_loader):
#             input_ids = batch['input_ids'].to(device)
#             attention_mask = batch['attention_mask'].to(device)
#             labels = batch['labels'].to(device)
#             outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
#             val_loss += outputs.loss.item()
#             preds.extend(torch.argmax(outputs.logits, dim=1).cpu().numpy())
#             true_labels.extend(labels.cpu().numpy())

#     # Compute metrics
#     avg_val_loss = val_loss / len(val_loader)
#     accuracy = accuracy_score(true_labels, preds)
#     f1 = f1_score(true_labels, preds, average='weighted')

#     print(f"Validation Loss: {avg_val_loss:.4f}")
#     print(f"Validation Accuracy: {accuracy:.4f}")
#     print(f"Validation F1 Score: {f1:.4f}")
#     print(classification_report(true_labels, preds))

#     # Early Stopping
#     if avg_val_loss < best_val_loss:
#         best_val_loss = avg_val_loss
#         patience_counter = 0
#         torch.save(model.state_dict(), 'best_model.pt')
#     else:
#         patience_counter += 1
#         if patience_counter >= patience:
#             print("Early stopping triggered!")
#             break

#     # Step the scheduler
#     scheduler.step()

#     # Clear GPU memory
#     torch.cuda.empty_cache()


  0%|          | 0/5135 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# """
# OG CODE

# """
# class BugDataset(Dataset):
#     def __init__(self, encodings, labels):
#         self.encodings = encodings
#         self.labels = labels

#     def __getitem__(self, idx):
#         item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
#         item['labels'] = torch.tensor(self.labels[idx])
#         return item

#     def __len__(self):
#         return len(self.labels)

# labels = df['severity_label'].values
# print(np.unique(labels))

# print(len(labels))
# dataset = BugDataset(tokenized_data, labels)

# # Split dataset into training and validation sets
# train_size = int(0.8 * len(dataset))
# val_size = len(dataset) - train_size
# train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# # Device setup
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=7)
# model.to(device)


[0 1 2 4 5 6]
51346


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e